In [1]:
import pandas as pd
import folium

In [10]:
folium_map_test = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles='cartodbpositron')
marker = folium.CircleMarker(location=[40.738, -73.98])
marker.add_to(folium_map_test)

In [11]:
folium_map_test

In [12]:
# locations.csvを取り込む
locations = pd.read_csv('locations.csv')
locations.head()

,start station id,start station latitude,start station longitude,start station name
0,72,40.767272,-73.993929,W 52 St & 11 Ave
1,79,40.719116,-74.006667,Franklin St & W Broadway
2,82,40.711174,-74.000165,St James Pl & Pearl St
3,83,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl
4,119,40.696089,-73.978034,Park Ave & St Edwards St


In [13]:
# データフレームを作る
departure_counts = pd.read_csv('departure_counts.csv')
arrival_counts = pd.read_csv('arrival_counts.csv')

In [14]:
departure_counts.head()

,start station id,departure count
0,72,230
1,79,124
2,82,67
3,83,44
4,119,54


In [15]:
arrival_counts.head()

,start station id,arrival count
0,72,229
1,79,127
2,82,60
3,83,42
4,119,62


In [16]:
# 出発と到着をData Frameとしてまとめる
trip_counts = departure_counts.merge(locations).merge(arrival_counts)

In [17]:
# こまめにデータを見てみる( again )
trip_counts.head()

,start station id,departure count,start station latitude,start station longitude,start station name,arrival count
0,72,230,40.767272,-73.993929,W 52 St & 11 Ave,229
1,79,124,40.719116,-74.006667,Franklin St & W Broadway,127
2,82,67,40.711174,-74.000165,St James Pl & Pearl St,60
3,83,44,40.683826,-73.976323,Atlantic Ave & Fort Greene Pl,42
4,119,54,40.696089,-73.978034,Park Ave & St Edwards St,62


In [20]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="cartodbpositron")

trip_counts["net departures"] = trip_counts["departure count"] - trip_counts["arrival count"]

for index, row in trip_counts.iterrows():
    try:
        net_departures = row["net departures"]
        net_departures = net_departures / 2 # 円の大きさを調整
        radius = abs(net_departures) # 純出発or到着数が多いほど円が大きくなるように半径を設定しておく

        popup_text = """{}<br>
                        total departures: {}<br>
                        total arrivals: {}<br>
                        net departures: {}"""
        popup_text = popup_text.format(row["start station name"],
                                       row["arrival count"],
                                       row["departure count"],
                                       net_departures)

        if net_departures > 0:
            color="#E37222" # オレンジ
        else:
            color="#0A8A9F" # ティール

        folium.CircleMarker(location=(row["start station latitude"], row["start station longitude"]),
                                                            radius=radius,
                                                            color=color,
                                                            fill=True,
                                                            popup=popup_text
                                           ).add_to(folium_map) # 最初に定義しているfolium_mapというオブジェクトにデータを加える
    except ValueError:
        print(index)

In [ ]:
folium_map

In [80]:
folium_map.save("./mapping_bike_data.html")